In [1]:
import pandas as pd
from Bio import SeqIO, Seq, Entrez
import numpy as np
import datetime
import ete3
import re
import math
import statistics
import random
import scipy
import h5py
import time
import os
import statsmodels.api as sm

import plotly.express as px


In [3]:
def calc_lognorm_CI(theseries, zscore):
    theseries = [math.log(x) for x in list(theseries)]
    CIlo = statistics.mean(theseries)  + (statistics.stdev(theseries)/2) + \
    zscore * math.sqrt( (statistics.stdev(theseries)/len(theseries)) + ( pow(statistics.stdev(theseries), 2) / (2 * (len(theseries)-1)) ) )

    return pow(math.e, CIlo)

In [4]:

def sum_entropy(entrdf_f, nodedetsdf_f, tfile, outnam):

    entrdf = pd.read_csv(entrdf_f,index_col=0)
    nodedetsdf = pd.read_csv(nodedetsdf_f)
    
    #load tree with node labels
    thet = ete3.Tree(tfile, format=1)   

    print(len(thet))
    entrdiffs = []

    #fill na for easier processing
    nodedetsdf = nodedetsdf.fillna('')
    
    
    counter = 0
    #for each node name
        #this will be the child node that acquired the mutations
    for cnod in thet.traverse():
        counter = counter + 1
        if counter%100 == 0:
            print(counter)#
        
        
        #get node above, this will be the context where the mutations happened
        upnod = cnod.up
        
        if upnod != None:
            
            if len(upnod.name) > 0:
            
                #ensure that the node is in the summarised dataframe
                if cnod.name in list(nodedetsdf.node):            
                    #get all the sites that have been substituted on that branch
                    thesites = [x[1:-1] for x in list(nodedetsdf[nodedetsdf.node == cnod.name].subs)[0].split(',') if len(x)>=3]

                    for site in thesites:

                        #list of entropies for that site excluding the parent name
                        nodexcl_entr = entrdf.loc[[x for x in list(entrdf.index) if x != upnod.name], site]
                        nodexcl_entr.dropna(inplace=True)

                        #need at least 3 seqs that don't have gaps (informative entr)
                        if len(nodexcl_entr) > 2:

                            entrmed = nodexcl_entr.median()
                            entrCI = calc_lognorm_CI(nodexcl_entr, 2.33)
                            # print(n.name, site)
                            siteentr = entrdf.loc[upnod.name, site]
                            entrdiffs.append([upnod.name, cnod.name, site, siteentr, entrmed, entrCI])
    
    rezdf = pd.DataFrame(entrdiffs)
    rezdf.columns = ['node', 'child', 'site', 'site-entr', 'median-entr', 'CI99-entr']

    #drop NA on entropy column -> these are indel regions where there's gaps and miscalled indel substitutions
    rezdf.dropna(subset='site-entr', inplace = True)
    
    rezdf.sort_values(by='node', inplace=True)

    
    #add entropies of random sites excluding the true ones
    randentr_l = []
    randmedentr_l = []

    for n in sorted(list(set(rezdf.node))):
        
        truesites = list(rezdf[rezdf.node == n].site)
        #dropna here -> i.e. exclude sites with indels in the whole alignment
        thisnentrdf = entrdf.dropna(axis=1)
        thisnentrdf = thisnentrdf[thisnentrdf.index == n]

        #randomly get an equal number of random sites that did not get substituted
        randsites =  random.sample([x for x in list(thisnentrdf.columns) if x not in truesites], len(truesites))
    
        for s in randsites:
            
            #get the entropy of the random site selected
            randentr_l.append(entrdf.loc[n, s])

            #list of entropies for that site excluding the parent name
            nodexcl_randentr = entrdf.loc[[x for x in list(entrdf.index) if x != n], s]
            nodexcl_randentr.dropna(inplace=True)
            #add the median for that site
            randmedentr_l.append(nodexcl_randentr.median())

    rezdf['rand-entr'] = randentr_l
    rezdf['rand-entr-med'] = randmedentr_l

    
    rezdf['cat'] = ['internal' if 'NODE_' in x else 'external' for x in list(rezdf.child)]
    rezdf['med-diff'] = rezdf['site-entr'] -  rezdf['median-entr']
    rezdf['CI99-diff'] = rezdf['site-entr'] -  rezdf['CI99-entr']
    
    rezdf['rand-med-diff'] = rezdf['rand-entr'] - rezdf['rand-entr-med']

    rezdf.to_csv(outnam, index=False)
    
    return rezdf
                    
                    

In [5]:
def plot_meddiffcomp(entrrezdf):
    entr_HAall_rVsv = entrrezdf.melt(value_vars=['med-diff', 'rand-med-diff'], value_name='entropy difference from the median')
    entr_HAall_rVsv = entr_HAall_rVsv.replace({'med-diff':'substituted site', 'rand-med-diff':'random site'})
    
    therange = [min(entrrezdf['med-diff'].quantile(0.25) , entrrezdf['rand-med-diff'].quantile(0.25))-0.1,
                max(entrrezdf['med-diff'].quantile(0.75) , entrrezdf['rand-med-diff'].quantile(0.75))+0.1]
    print(therange)
    
    fig = px.box(entr_HAall_rVsv, y='entropy difference from the median', color='variable', points=False, range_y=therange, color_discrete_sequence = palette5 )
#     fig = px.box(entr_HAall_rVsv, y='entropy difference from the median', color='variable', points=False, color_discrete_sequence = palette5 )
    fig.add_hline(0, line_width=1, line_dash="dash", line_color="grey")
    fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
    fig.update_traces(boxmean=True)
     
    fig.show()   

<br>
<br>

## Summarise pLM entropies per substituted site


In [4]:

for sero in ['gisaid_h7_270624_filt', 'gisaid_h5_270624_filt', 'ncbi_h1_110424_filt', 'ncbi_h3n2_110424_filt']:

    print(sero)
    
    for entrfile in os.listdir('data/%s/site-entropy-aln/'%sero):
        
        mod = entrfile.replace('%s-'%sero, '').replace('-site_entropy_aln.csv', '')
    
        print('\n' + mod)
        
        sum_entropy('data/%s/site-entropy-aln/%s'%(sero,entrfile), 
                'data/%s/%s_nodedetails.csv'%(sero,sero),
                'data/%s/%s_tf.nwk'%(sero,sero),
                'data/%s/entr-rez/%s_tf-%s_entr-rez.csv'%(sero, sero, mod))

<br>
<br>

## Plotting difference between substituted and random sites' pLM entropies

In [ ]:
def plot_meddiffcomp(entrrezdf, outf):
    entr_HAall_rVsv = entrrezdf.melt(value_vars=['med-diff', 'rand-med-diff'], value_name='entropy difference from the median')
    entr_HAall_rVsv = entr_HAall_rVsv.replace({'med-diff':'substituted site', 'rand-med-diff':'random site'})
    
    therange = [min(entrrezdf['med-diff'].quantile(0.25) , entrrezdf['rand-med-diff'].quantile(0.25))-0.1,
                max(entrrezdf['med-diff'].quantile(0.75) , entrrezdf['rand-med-diff'].quantile(0.75))+0.1]

    qdiff = 0
    #if sub is fully contained in the nonsub qdiff = 100
    if (entrrezdf['rand-med-diff'].quantile(0.75) > entrrezdf['med-diff'].quantile(0.75)) and (entrrezdf['rand-med-diff'].quantile(0.25) < entrrezdf['med-diff'].quantile(0.25)):
        qdiff = 100
    elif entrrezdf['med-diff'].quantile(0.25) > entrrezdf['rand-med-diff'].quantile(0.75):
        qdiff = 0
    else:
        #the percentage of the med-diff IQR that's under the rand-med-diff 75% IQR point
        qdiff = (entrrezdf['rand-med-diff'].quantile(0.75) - entrrezdf['med-diff'].quantile(0.25)) / (entrrezdf['med-diff'].quantile(0.75) - entrrezdf['med-diff'].quantile(0.25))
        qdiff = int(qdiff*100)
    
    print('Difference between top and bottom quartile: ',
          qdiff,
          entrrezdf['med-diff'].mean() - entrrezdf['rand-med-diff'].mean(), 
         scipy.stats.skew(entrrezdf['med-diff']),
         scipy.stats.skew(entrrezdf['rand-med-diff']))
    
    
    fig = px.box(entr_HAall_rVsv, y='entropy difference from the median', color='variable', points=False, range_y=therange, color_discrete_sequence = palette5 )
    fig.add_hline(0, line_width=1, line_dash="dash", line_color="grey")
    fig.update_layout(
        {'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)'},
        showlegend=False,
        title={ 'text': str(qdiff) + " %", 'x': 0.5, 'xanchor': 'center' },
        yaxis_title = None)
    
    fig.update_traces(boxmean=True)
    fig.write_image(outf, width=300, height=300)


In [ ]:
for ser in ['gisaid_h7_270624_filt', 'gisaid_h5_270624_filt', 'ncbi_h1_110424_filt', 'ncbi_h3n2_110424_filt']:
    
    for x in os.listdir('data/%s/entr-rez/'%ser):

        if ('H1' not in x) and ('H5' not in x) and ('H7' not in x) and ('H3' not in x) and ('8020' not in x):
            print(x)
            entrrezdf = pd.read_csv('data/%s/entr-rez/'%ser + x)
            plot_meddiffcomp(entrrezdf, 'plotting/fig3/' + x.replace('_entr-rez.csv', '_boxes.svg'))

In [ ]:
def plot_meddiffcomp80(entrrezdf, outf, datfile):
    
    metadf = pd.read_csv(datfile)
    entrrezdf80 = entrrezdf[entrrezdf.node.isin(metadf[metadf.date>2015.16].node)]
    
    entr_HAall_rVsv = entrrezdf80.melt(value_vars=['med-diff', 'rand-med-diff'], value_name='entropy difference from the median')
    entr_HAall_rVsv = entr_HAall_rVsv.replace({'med-diff':'substituted site', 'rand-med-diff':'random site'})
    
    therange = [min(entrrezdf80['med-diff'].quantile(0.25) , entrrezdf80['rand-med-diff'].quantile(0.25))-0.1,
                max(entrrezdf80['med-diff'].quantile(0.75) , entrrezdf80['rand-med-diff'].quantile(0.75))+0.1]
   
    qdiff = 0
    #if sub is fully contained in the nonsub qdiff = 100
    if (entrrezdf80['rand-med-diff'].quantile(0.75) > entrrezdf80['med-diff'].quantile(0.75)) and (entrrezdf80['rand-med-diff'].quantile(0.25) < entrrezdf80['med-diff'].quantile(0.25)):
        qdiff = 100
    elif entrrezdf80['med-diff'].quantile(0.25) > entrrezdf80['rand-med-diff'].quantile(0.75):
        qdiff = 0
    else:
        #the percentage of the med-diff IQR that's under the rand-med-diff 75% IQR point
        qdiff = (entrrezdf80['rand-med-diff'].quantile(0.75) - entrrezdf80['med-diff'].quantile(0.25)) / (entrrezdf80['med-diff'].quantile(0.75) - entrrezdf80['med-diff'].quantile(0.25))
        qdiff = int(qdiff*100)
    
    
    print('Difference between top and bottom quartile: ', qdiff)
    
    
    fig = px.box(entr_HAall_rVsv, y='entropy difference from the median', color='variable', points=False, range_y=therange, color_discrete_sequence = palette5 )
    fig.add_hline(0, line_width=1, line_dash="dash", line_color="grey")
    fig.update_layout(
        {'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)'},
        showlegend=False,
        title={ 'text': str(qdiff) + " %", 'x': 0.5, 'xanchor': 'center' },
        yaxis_title = None)
    
    fig.update_traces(boxmean=True)
    fig.write_image(outf, width=300, height=300)


In [ ]:
for ser in ['gisaid_h7_270624_filt', 'gisaid_h5_270624_filt', 'ncbi_h1_110424_filt', 'ncbi_h3n2_110424_filt']:
    
    for x in os.listdir('data/%s/entr-rez/'%ser):

        if '8020' in x:
            print(x)
            entrrezdf = pd.read_csv('data/%s/entr-rez/'%ser + x)
            plot_meddiffcomp80(entrrezdf, 'plotting/fig3/' + x.replace('_entr-rez.csv', '_boxes.svg'), 'data/%s/%s_nodedetails.csv'%(ser, ser))

<br>
<br>

## Logistic regression

In [43]:
#do original model and HA-all


for sero in ['gisaid_h7_270624_filt',
             'gisaid_h5_270624_filt',
             'ncbi_h1_110424_filt',
             'ncbi_h3n2_110424_filt']:

    for mod in [
        'T5_uniref_HA_221124',
        'T5_UniRef_e2',
        'esm2_t33_e10',
        'esm2_t33']:

        #read the file with all pLM entropies
        entrdf = pd.read_csv('data/%s/site-entropy-aln/%s-ASR-%s-site_entropy_aln.csv'%(sero, sero, mod), index_col=0)
        #and the file with the substituted nodes
        entrezdf = pd.read_csv('data/%s/entr-rez/%s_tf-ASR-%s_entr-rez.csv'%(sero, sero, mod))

        logregrez = []

        #for each sequence:
        for x in set(entrezdf.node):

            #get the sites substituted in that internal node sequence
            subsites = set(entrezdf[entrezdf.node ==x].site)

            #only run the regression if there are more than 3 sites substituted
            if len(subsites)>2:

                #get pLM entropies for all sites of that sequence
                thisseqdf = entrdf.loc[x]
                thisseqdf = pd.DataFrame(thisseqdf.T)

                #annotate substituted (1) and non-substituted sites (0)
                thisseqdf['sub'] = [1 if int(n) in subsites else 0 for n in list(thisseqdf.index)]

                thisseqdf.columns = ['entr', 'sub']

                thisseqdf.dropna(subset='entr', inplace=True)

                #fit logistic regression
                X = thisseqdf['entr']
                y = thisseqdf['sub']
                X = sm.add_constant(X)

                model = sm.Logit(y,X)
                result = model.fit(method='bfgs')

                pseudo_r2 = 1 - (result.llf / result.llnull)    

                slope = result.params['entr']

                logregrez.append([x, result.llr_pvalue, pseudo_r2, slope])

        logregdf = pd.DataFrame(logregrez)

        logregdf.columns = ['node', 'pval', 'pseudoR2', 'slope']

        logregdf.to_csv('data/%s/logreg/%s_tf-ASR-%s_logreg-pernode-rez.csv'%(sero, sero, mod),index=False)
    
    
    

In [2]:
#do alignment


for sero in ['gisaid_h7_270624_filt',
             'gisaid_h5_270624_filt',
             'ncbi_h1_110424_filt',
             'ncbi_h3n2_110424_filt']:

    seqdic = SeqIO.to_dict(SeqIO.parse('data/%s/ancestral_sequences.fasta'%sero, 'fasta'))
    
    #remove internal node seqs from per site entropy calculations
    aldf = pd.DataFrame({x:[a for a in seqdic[x]] for x in list(seqdic) if 'NODE_' not in x})
    
    entrezdf = pd.read_csv('data/%s/entr-rez/%s_tf-ASR-esm2_t33_e10_entr-rez.csv'%(sero, sero))
    
    
    #make MSA entropy list
    alentropy = []
    
    for i in range(len(aldf)):
        reslist = aldf.iloc[i].tolist()
    #     print(reslist)
        if '-' in reslist:
            reslist.remove('-')
        proplist = [reslist.count(x)/len(reslist) for x in set(reslist)]
        #calculate alignment site entropy
        alentropy.append(scipy.stats.entropy(proplist, base=2))

        
        

    logregrez = []

    #for each sequence:
    for x in set(entrezdf.node):

        subsites = set(entrezdf[entrezdf.node ==x].site)

        if len(subsites)>2:

            thisseqdf = pd.DataFrame(alentropy)

            thisseqdf['sub'] = [1 if int(n) in subsites else 0 for n in list(thisseqdf.index)]

            thisseqdf.columns = ['entr', 'sub']

            thisseqdf.dropna(subset='entr', inplace=True)

    #         print(subsites)
    #         print(thisseqdf.sort_values(by='sub'))


            #regression
            X = thisseqdf['entr']
            y = thisseqdf['sub']
            X = sm.add_constant(X)

            model = sm.Logit(y,X)
            result = model.fit(method='bfgs')

            pseudo_r2 = 1 - (result.llf / result.llnull)    

    #         thisseqdf['logreg'] = result.predict(X)

    #         fig = px.scatter(thisseqdf, x='entr', y=['sub', 'logreg'] )
    #         fig.show()

            slope = result.params['entr']

            logregrez.append([x, result.llr_pvalue, pseudo_r2, slope])

    logregdf = pd.DataFrame(logregrez)

    logregdf.columns = ['node', 'pval', 'pseudoR2', 'slope']

    logregdf.to_csv('data/%s/logreg/%s_tf-MSA_logreg-pernode-rez.csv'%(sero, sero),index=False)

    

    